In [1]:
## Import necessary libraries
# --- Core numerical & data handling ---
import numpy as np
import pandas as pd
import geopandas as gpd
import netCDF4 as nc

# --- Raster & geospatial ---
import rasterio # rioxarray extends xarray to understand geospatial rasters (GeoTIFF, NetCDF…)
from rasterio.transform import from_origin
import rioxarray as rxr
import geojson as gj
from affine import Affine

# --- Plotting & visualization ---
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Rectangle, Circle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# --- Cartopy (mapping) ---
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# --- Utilities ---
import datetime
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2025-09-16 15:12:33


## Loading the GeoTiff data

In [ ]:

'''
Loading MWIR data from Your_data.tif

F002 - Mission 
L1 - Level 1 processing
IR - Infrared
L2L1M0 - Level 2 processing, Level 1 data, Mode 0
 - Start time of data acquisition
 - End time of data acquisition
- Unique identifier for the data file
MWIR - Mid-Wave Infrared band

'''

### Analogus to fits files stored in a directory loading the GeoTiFF file
# Open a GeoTIFF file with rioxarray


da = rxr.open_rasterio("Dataset_1/Your_data.tif").squeeze() # Remove the band dimension if it's single-band
print("Original coordinate reference system:", da.rio.crs) # Coordinate Reference System
print("Original resolution:", da.rio.resolution()) # (pixel width, pixel height)
print("Original bounds:", da.rio.bounds()) # (left, bottom, right, top) in the CRS units
print("Original shape:", da.shape) # (height, width)

print()

In [ ]:
# Another way to load data
path = "Dataset_1/Your_data.tif"

with rasterio.open(path) as ds:
    print("CRS:", ds.crs)
    print("Resolution:", ds.res)
    print("Bounds:", ds.bounds)
    print("Dtypes:", ds.dtypes)                 # per-band dtype
    print("Nodata:", ds.nodatavals)             # per-band NoData
    print("Descriptions:", ds.descriptions)     # per-band descriptions (names)
    print("Global tags:", ds.tags())            # file-level tags/metadata
    print("Band-1 tags:", ds.tags(1))           # band-level tags (often where units live)
    print("Scales:", ds.scales)                 # scale_factor (if any)
    print("Offsets:", ds.offsets)               # add_offset (if any)
    print("Profile:", ds.profile)               # compact summary


with rasterio.open(path) as ds:
    print("Number of bands:", ds.count)        # how many channels
    print("Descriptions:", ds.descriptions)    # names/labels of each band
    for i in range(1, ds.count + 1):
        print(f"Band {i} dtype:", ds.dtypes[i-1])
        print(f"Band {i} tags:", ds.tags(i))   # metadata per band


In [ ]:
# This shows all pixel values, including invalid NoData fill values (huge negatives).
arr_pre_mask = da.values

lo, hi = np.nanpercentile(arr_pre_mask, 1), np.nanpercentile(arr_pre_mask, 80)

print(f"Data range before masking: {lo} to {hi}")
plt.imshow(da.values, cmap="inferno",vmin=2, vmax=2.5) # Units are W/m^2/sr/um for radiance
plt.colorbar(label="Pixel values")
plt.title("MWIR Data before masking (includes fill values)")

plt.savefig("Plots/Dataset_1/Your_data_before_masking.png", dpi=300)

In [ ]:
## As the mask value is already applied to the data set at -3.4e38 we can use this to mask out the invalid values

# Mask out NoData and non-finite values
da_clean = da.where(np.isfinite(da) & (da != da.rio.nodata))

# Optional: mask negative radiance (physically invalid)
da_clean = da_clean.where(da_clean >= 0)

arr = da_clean.values
print("Shape:", arr.shape)
print("Min:", np.nanmin(arr))
print("P02:", np.nanpercentile(arr, 2))
print("Median:", np.nanmedian(arr))
print("P98:", np.nanpercentile(arr, 98))
print("Max:", np.nanmax(arr))
print("NaN fraction:", np.mean(~np.isfinite(arr)))
print("Units:", da.attrs.get("units"))




In [ ]:

# Percentile stretch for background detail
lo, hi = np.nanpercentile(arr, 1), np.nanpercentile(arr, 90)

# Create side-by-side figure
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# --- Plot 1: MWIR image (robust stretch) ---
im = axes[0].imshow(arr, vmin=lo, vmax=hi, cmap="inferno")
cbar = plt.colorbar(im, ax=axes[0], fraction=0.046, pad=0.04)
cbar.set_label("MWIR radiance (approx)", fontsize=10)

axes[0].set_title("MWIR Level-1 — cleaned", fontsize=12, loc="center")
axes[0].axis("on")  # cleaner without axes

# --- Plot 2: Histogram ---
vals = arr[np.isfinite(arr)]
axes[1].hist(vals, bins=200, log=True, range=(0, 70), color="Blue", edgecolor="blue",alpha = 0.5)
axes[1].set_xlabel("MWIR pixel value", fontsize=10)
axes[1].set_ylabel("Count (log scale)", fontsize=10)
axes[1].set_title("MWIR histogram — background vs hotspots", fontsize=12, loc="center")
axes[1].grid(True, which="both", linestyle="--", linewidth=0.5, alpha=0.7)


# Tidy layout
plt.tight_layout()
plt.savefig("Plots/Dataset_1/MWIR_combined.png", dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
print(np.where(np.nanmax(arr) == arr)) # Find the location of the maximum value in the array
print(arr[1400:1500,800:1000]) # Find the pixel values around the maximum value

In [ ]:

# Existing subset + stretch
sub_arr = arr[1400:1480, 950:1000]
lo, hi = np.nanpercentile(sub_arr, 1), np.nanpercentile(sub_arr, 99.999)

# Find brightest pixel (row r, col c) to plot a circle around
r, c = np.unravel_index(np.nanargmax(sub_arr), sub_arr.shape)

fig, ax = plt.subplots()
im = ax.imshow(sub_arr, vmin=lo, vmax=hi, cmap="plasma")
plt.colorbar(im, ax=ax, label="MWIR radiance (assumed units)")
ax.set_title("MWIR Subset (rows 1400–1480, cols 950–1000)")

# Draw a circle (tweak radius if needed)
ax.add_patch(Circle((c, r), radius=5, fill=False, edgecolor="white", linewidth=2))
ax.annotate(f"peak={sub_arr[r, c]:.1f}", (c+6, r), color="white", fontsize=9)
plt.grid()
plt.savefig("Plots/Dataset_1/MWIR_Zoomed.png", dpi=300,bbox_inches='tight')
plt.show()


## Load the geojson data

In [ ]:
# --- Load detections from GeoJSON ---
# Contains candidate detection points (e.g., hotspots). 
# Each detection has geographic coordinates in the file’s CRS.
gdf = gpd.read_file(
    "Dataset_1/Your_data.geojson"
)

# Reproject detections to the raster’s CRS so both are aligned
# (da.rio.crs is the CRS of the raster opened with rioxarray).
gdf = gdf.to_crs(da.rio.crs)  
print(gdf.head(160))


# --- Map world coordinates → pixel indices (col, row) ---
# The raster stores data in a grid. We need to map each detection’s (x,y) position 
# into grid indices so we can extract raster values underneath them.

A = da.rio.transform()  
# Affine transformation matrix from raster metadata. 
# It defines the mapping: (col,row) → (x,y) in world coordinates.

A = Affine(*A[:6]) if not isinstance(A, Affine) else A   # 2D affine described by 6 parameters
# Ensure we have a proper Affine object.
# Affine encodes transforms of the form:
#   x = A*col + B*row + C
#   y = D*col + E*row + F

# Invert the transform: now we can map (x,y) → (col,row).
# This is what we need to find the pixel each detection falls into.

Ainv = ~A  

# Print few world coordinates
print("Geometric x and y values = ", gdf.geometry.x.values[:5], gdf.geometry.y.values[:5])

# --- Convert detection coordinates to pixel indices ---
# Apply inverse affine transform to get fractional (col, row) positions.
cols, rows = Ainv * (gdf.geometry.x.values, gdf.geometry.y.values)
print("Sample cols, rows:", cols[:5], rows[:5])

# Round to nearest integer pixel indices.
# NOTE: rows = y-direction (height), cols = x-direction (width).
cols = np.round(cols).astype(int)
rows = np.round(rows).astype(int)


# --- Clip to valid raster bounds ---
H, W = arr.shape
ok = (rows >= 0) & (rows < H) & (cols >= 0) & (cols < W)

# Assign row/col values; invalid indices → -1.
gdf["row"] = np.where(ok, rows, -1)
gdf["col"] = np.where(ok, cols, -1)


# Extract raster MWIR values at those pixel indices
# If detection falls outside raster, assign NaN.
gdf["mwir_val"] = np.where(ok, arr[rows[ok], cols[ok]], np.nan)



# --- Output ---
print()
print("Total detections:", len(gdf))
print(gdf[["row", "col", "mwir_val"]].head())
print("Valid MWIR detections:", gdf["mwir_val"].notna().sum())


1. [Data Base for maps of Region](https://databasin.org/maps/new/#datasets=3fac6542263d4972af2f55dc13737f36)

In [ ]:


# Extract lat/lon separately from the GeoDataFrame
lons = gdf.geometry.x.values
lats = gdf.geometry.y.values

# Plot map using Cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig, ax = plt.subplots(figsize=(9,9), subplot_kw={'projection': ccrs.PlateCarree()})

# Add coastlines and borders
ax.coastlines(resolution="10m")
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor="gray")

# Scatter points
ax.scatter(lons, lats, color="red", s=60, transform=ccrs.PlateCarree(), label="Given Coordinates")

# Set extent to Region region
ax.set_extent([], crs=ccrs.PlateCarree()) # 

ax.set_title("Map of Provided Coordinates Region", fontsize=14) # as given in json file
ax.legend()
plt.show()


In [ ]:


# ------------------------------------------------------------
# Extract coordinates and MWIR values from the GeoDataFrame
# ------------------------------------------------------------
lons = gdf.geometry.x.values
lats = gdf.geometry.y.values
mwir_vals = gdf["mwir_val"].values

# ------------------------------------------------------------
# Normalize MWIR values to scale transparency (alpha)
# Ensures faint background detections are visible but strong
# hotspots stand out more clearly
# ------------------------------------------------------------
norm = (mwir_vals - mwir_vals.min()) / (mwir_vals.max() - mwir_vals.min())
alphas = 0.3 + 0.7 * norm   # alpha in [0.3, 1.0]

# ------------------------------------------------------------
# Create side-by-side figure with two Cartopy maps
# ------------------------------------------------------------
fig, axes = plt.subplots(
    1, 2, figsize=(16, 7), subplot_kw={'projection': ccrs.PlateCarree()}
)

# ============================================================
# LEFT PANEL: Raw coordinate scatter (all detections in red)
# ============================================================
ax = axes[0]
ax.coastlines(resolution="10m")
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor="gray")

# Plot points (red, fixed size)
ax.scatter(
    lons, lats, 
    color="red", s=60,
    transform=ccrs.PlateCarree(),
    label="Given Coordinates"
)

# Zoom in on Region
ax.set_extent([*coordinates of region*], crs=ccrs.PlateCarree())

ax.set_title("Map of Provided Coordinates\n",
             fontsize=13, loc="center")
ax.legend()

# ============================================================
# RIGHT PANEL: MWIR detections (colored by intensity)
# ============================================================
ax = axes[1]
ax.coastlines(resolution="10m")
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle=':', edgecolor="gray")

# Scatter points, colored by MWIR value + transparency scaled
sc = ax.scatter(
    lons, lats, 
    c=mwir_vals, cmap="plasma",    
    s=100, alpha=alphas,
    transform=ccrs.PlateCarree()
)

cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.8, pad=0.05)
cbar.set_label("MWIR Intensity", fontsize=11)

# Same region extent
ax.set_extent([coordinates for region], crs=ccrs.PlateCarree())

ax.set_title("MWIR Detections\n",
             fontsize=13, loc="center")

# ------------------------------------------------------------
# Save and show the combined figure
# ------------------------------------------------------------
plt.tight_layout()
plt.savefig("Plots/Dataset_1/Coordinates_vs_MWIR.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:

row_min, row_max = 1400, 1480
col_min, col_max = 950, 1000


# Create a mask for detections inside this window
inside = (
    (gdf["row"] >= row_min) & (gdf["row"] < row_max) &
    (gdf["col"] >= col_min) & (gdf["col"] < col_max)
)


# Filter GeoDataFrame
gdf_sub = gdf[inside].copy()


# Print detections in this subwindow with their MWIR values
print(gdf_sub[["row", "col", "mwir_val"]])


row_pix = gdf_sub["row"]
col_pix = gdf_sub["col"]
mwir_vals = gdf_sub["mwir_val"]


# Extract data
row_pix = gdf_sub["row"].values # pixel row indices (y-axis in raster)
col_pix = gdf_sub["col"].values # pixel col indices (y-axis in raster)
x = gdf_sub.geometry.x.values   # ongitude in degrees (EPSG:4326)
y = gdf_sub.geometry.y.values   # latitude in degrees (EPSG:4326)
mwir_vals = gdf_sub["mwir_val"].values

# --- Colormap setup ---
# Copy rainbow colormap and force values below vmin to show as white
cmap = plt.cm.rainbow.copy()
cmap.set_under("white")

# --- Create side-by-side plots ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# =====================================================
# Left panel: Pixel space
# =====================================================
hist1 = axes[0].hist2d(
    col_pix, row_pix,
    weights=mwir_vals,
    bins=(50, 50),
    cmap=cmap,
    vmin=0.01
)
fig.colorbar(hist1[3], ax=axes[0], label="MWIR intensity")
axes[0].set_xlabel("Column (pixels)")
axes[0].set_ylabel("Row (pixels)")
axes[0].set_title("Detections in Pixel Space")
axes[0].ticklabel_format(style="plain")
axes[0].grid(True, linestyle="--", alpha=0.5)  # add grid overlay


# =====================================================
# Right panel: Geographic space (lon/lat)
# =====================================================

# --- Right: Geographic / CRS space ---
hist2 = axes[1].hist2d(
    x, y,
    weights=mwir_vals,
    bins=(50, 50),
    cmap=cmap,
    vmin=0.01
)
fig.colorbar(hist2[3], ax=axes[1], label="MWIR intensity")
axes[1].set_xlabel("Longitude / X coordinate")
axes[1].set_ylabel("Latitude / Y coordinate")
axes[1].set_title("Detections in CRS Space")
axes[1].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
axes[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
axes[1].grid(True, linestyle="--", alpha=0.5)  # add grid overlay

plt.tight_layout()
plt.show()


In [ ]:

# --- Data ---
lons = gdf_sub.geometry.x.values
lats = gdf_sub.geometry.y.values
mwir_vals = gdf_sub["mwir_val"].values

# --- Data-driven zoom bounds (for the inset) ---
pad = 0.01  # degrees
lon_min, lon_max = float(np.nanmin(lons) - pad), float(np.nanmax(lons) + pad)
lat_min, lat_max = float(np.nanmin(lats) - pad), float(np.nanmax(lats) + pad)

# --- Alpha scaling ---
mv_min, mv_max = np.nanmin(mwir_vals), np.nanmax(mwir_vals)
if not np.isfinite(mv_max - mv_min) or (mv_max - mv_min) == 0:
    alphas = np.full_like(mwir_vals, 0.7, dtype=float)
else:
    norm = (mwir_vals - mv_min) / (mv_max - mv_min + 1e-12)
    alphas = 0.3 + 0.7 * np.clip(norm, 0, 1)

# --- Figure & MAIN AXES ---
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': proj})

# Basemap
ax.add_feature(cfeature.LAND, facecolor="#b2df8a")
ax.add_feature(cfeature.OCEAN, facecolor="#87CEEB")
ax.coastlines(resolution="10m", linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linestyle=":", linewidth=0.7, edgecolor="#666666")
ax.add_feature(cfeature.STATES, linestyle=":", edgecolor="#888888", linewidth=0.6)

# Optional density underlay
hb = ax.hexbin(
    lons, lats, gridsize=40, bins="log", mincnt=1,
    transform=proj, cmap="Greys", alpha=0.5
)

# Points
sc = ax.scatter(
    lons, lats, c=mwir_vals, cmap="plasma",
    s=90, linewidths=0.6, edgecolors="k",
    alpha=alphas, transform=proj, zorder=5
)

# Colorbar
cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.7, pad=0.03)
cbar.set_label("MWIR Intensity", fontsize=11)

# Gridlines
gl = ax.gridlines(draw_labels=True, linewidth=0.6, color="gray", alpha=0.55, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.MultipleLocator(0.5)
gl.ylocator = mticker.MultipleLocator(0.5)
gl.xlabel_style = {"size": 9}
gl.ylabel_style = {"size": 9}

# MAIN extent (keep wide SoCal view)
ax.set_extent([], crs=proj)

# --- Draw inset rectangle on MAIN map (shows the zoom area) ---
rect = Rectangle(
    (lon_min, lat_min), (lon_max - lon_min), (lat_max - lat_min),
    linewidth=1.5, edgecolor="black", facecolor="none", linestyle="-"
)
ax.add_patch(rect)
rect.set_transform(proj)  # important for geodata coords

# --- INSET AXES (zoom) ---
# Position given in axes fraction coordinates [left, bottom, width, height]
# Tweak as you like (e.g., loc bottom-right)
from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# --- INSET AXES ---
inset_ax = inset_axes(
    ax,
    width="38%", height="38%", loc="lower right", borderpad=1.2,
    axes_class=GeoAxes,  # <- fixed
    axes_kwargs=dict(map_projection=proj)
)

# Inset basemap (lighter lines so points pop)
inset_ax.add_feature(cfeature.LAND, facecolor="#d9f0c1")
inset_ax.add_feature(cfeature.OCEAN, facecolor="#bfe3f6")
inset_ax.coastlines(resolution="10m", linewidth=0.6)
inset_ax.add_feature(cfeature.STATES, linestyle=":", edgecolor="#666666", linewidth=0.5)

# Inset plots (same data but tighter extent)
inset_ax.hexbin(
    lons, lats, gridsize=30, bins="log", mincnt=1,
    transform=proj, cmap="Greys", alpha=0.4, zorder=2
)
inset_ax.scatter(
    lons, lats, c=mwir_vals, cmap="plasma",
    s=70, edgecolors="k", linewidths=0.5, alpha=alphas,
    transform=proj, zorder=3
)
inset_ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=proj)

# Optional inset gridlines (no labels to keep it clean)
igl = inset_ax.gridlines(draw_labels=False, linewidth=0.4, color="gray", alpha=0.4, linestyle="--")

# --- North arrow on MAIN (uses MAIN extent) ---
nx = -* + 0.92 * ((*) - (-121))
ny = * + 0.15 * (*)
ax.text(nx, ny + 0.055 * (35.2 - 31.8), "N", transform=proj,
        ha="center", va="center", fontsize=12, fontweight="bold")
ax.annotate("",
            xy=(nx, ny + 0.05 * (35.2 - 31.8)), xytext=(nx, ny),
            xycoords=proj._as_mpl_transform(ax),
            textcoords=proj._as_mpl_transform(ax),
            arrowprops=dict(arrowstyle="-|>", lw=1.8, color="k"))

# Title
ax.set_title("MWIR Detections — (Inset: Data-driven Zoom)", fontsize=14, pad=10)

plt.tight_layout()
plt.savefig("Plots/Dataset_1/**.png", dpi=300, bbox_inches="tight")

plt.show()


## Additional backup stuff

In [ ]:
# import re
# from datetime import datetime

# path = "Dataset_1/Your_data.tif"

# pattern = r"(\d{4}-\d{2}-\d{2}T\d{6}\.\d+Z)"
# times = re.findall(pattern, path)

# start_time = datetime.strptime(times[0], "%Y-%m-%dT%H%M%S.%fZ")
# end_time   = datetime.strptime(times[1], "%Y-%m-%dT%H%M%S.%fZ")

# print("Start:", start_time)
# print("End:", end_time)


In [ ]:
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

# # Extract lon, lat, and mwir values from filtered gdf_sub
# lons = gdf.geometry.x.values
# lats = gdf.geometry.y.values
# mwir_vals = gdf["mwir_val"].values

# # Plot map using Cartopy
# fig, ax = plt.subplots(figsize=(9, 9), subplot_kw={'projection': ccrs.PlateCarree()})

# # Add coastlines, borders, states
# ax.coastlines(resolution="10m")
# ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.add_feature(cfeature.STATES, linestyle=':', edgecolor="gray")

# # Compute bounds with a small padding
# lon_min, lon_max = lons.min() - 0.5, lons.max() + 0.5
# lat_min, lat_max = lats.min() - 0.5, lats.max() + 0.5


# # Normalize MWIR values to [0.3, 1.0] for alpha (avoid fully invisible points)
# norm = (mwir_vals - mwir_vals.min()) / (mwir_vals.max() - mwir_vals.min())
# alphas = 0.3 + 0.7 * norm   # min alpha=0.3, max alpha=1.0

# sc = ax.scatter(
#     lons, lats, 
#     c=mwir_vals, 
#     cmap="plasma",    # Color by MWIR intensity
#     s=100, 
#     alpha=alphas,           # <--- per-point transparency
#     transform=ccrs.PlateCarree()
# )
# # Add colorbar for MWIR intensity
# cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.7, pad=0.05)
# cbar.set_label("MWIR Intensity", fontsize=12)

# # Set extent to Region
# ax.set_extent([], crs=ccrs.PlateCarree())
# # Apply zoomed extent
# # ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# ax.set_title("MWIR Detections on Map ", fontsize=14)

# plt.show()


In [ ]:
# # Define the subarray window
# row_min, row_max = 1400, 1480
# col_min, col_max = 950, 1000



# import matplotlib.pyplot as plt
# from shapely.geometry import Point

# # Coordinates provided by user (lat, lon)


# # Extract lat/lon separately 
# lons = gdf_sub.geometry.x.values
# lats = gdf_sub.geometry.y.values

# # Plot map using Cartopy
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

# fig, ax = plt.subplots(figsize=(9,9), subplot_kw={'projection': ccrs.PlateCarree()})

# # Add coastlines and borders
# ax.coastlines(resolution="10m")
# ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.add_feature(cfeature.STATES, linestyle=':', edgecolor="gray")

# # Scatter points
# ax.scatter(lons, lats, color="red", s=60, transform=ccrs.PlateCarree(), label="Given Coordinates")

# # Set extent to Region
# ax.set_extent([], crs=ccrs.PlateCarree())

# ax.set_title("Map of Provided Coordinates ", fontsize=14) # as given in json file
# ax.legend()
# plt.show()


In [56]:
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

# # Extract lon, lat, and mwir values from filtered gdf_sub
# lons = gdf_sub.geometry.x.values
# lats = gdf_sub.geometry.y.values
# mwir_vals = gdf_sub["mwir_val"].values

# # Plot map using Cartopy
# fig, ax = plt.subplots(figsize=(9, 9), subplot_kw={'projection': ccrs.PlateCarree()})

# # Add coastlines, borders, states
# ax.coastlines(resolution="10m")
# ax.add_feature(cfeature.BORDERS, linestyle=':')
# ax.add_feature(cfeature.STATES, linestyle=':', edgecolor="gray")

# # Compute bounds with a small padding
# lon_min, lon_max = lons.min() - 0.5, lons.max() + 0.5
# lat_min, lat_max = lats.min() - 0.5, lats.max() + 0.5


# # Normalize MWIR values to [0.3, 1.0] for alpha (avoid fully invisible points)
# norm = (mwir_vals - mwir_vals.min()) / (mwir_vals.max() - mwir_vals.min())
# alphas = 0.3 + 0.7 * norm   # min alpha=0.3, max alpha=1.0

# sc = ax.scatter(
#     lons, lats, 
#     c=mwir_vals, 
#     cmap="cividis_r", 
#     s=100, 
#     alpha=alphas,           # <--- per-point transparency
#     transform=ccrs.PlateCarree()
# )
# # Add colorbar for MWIR intensity
# cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.7, pad=0.05)
# cbar.set_label("MWIR Intensity", fontsize=12)

# # Set extent to Region
# ax.set_extent([[], crs=ccrs.PlateCarree())
# # Apply zoomed extent
# # ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# ax.set_title("MWIR Detections on Map", fontsize=14)

# plt.show()
